In [1]:
# Importing necessary libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import PolynomialFeatures
from sklearn.impute import SimpleImputer
from sklearn.model_selection import train_test_split,GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn import tree

from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, confusion_matrix, classification_report

In [2]:
# Load the dataset
data = pd.read_csv('C:/Users/Vidhya/Desktop/Study/Semester 1/ML/Assignments/weatherAUS.csv')


In [3]:
# # Drop unnecessary columns
# #data.drop('Unnamed: 0', axis=1, inplace=True)

# # Handle missing values
# data.dropna(inplace=True)

# Convert date column to datetime
data['Date'] = pd.to_datetime(data['Date'],format='%d-%m-%Y')
data['Date'] = data['Date'].values.astype("float64")

In [4]:
# Convert categorical columns to numerical using label encoding
categorical_cols = ['Location', 'WindGustDir', 'WindDir9am', 'WindDir3pm', 'RainToday', 'RainTomorrow']
label_encoder = LabelEncoder()
for col in categorical_cols:
    data[col] = label_encoder.fit_transform(data[col])

In [5]:
# Split the dataset into features and target variable
X = data.drop('RainTomorrow', axis=1)  # Features
y = data['RainTomorrow']  # Target variable

In [6]:
# Create new features
# Extract year, month, and day from the 'Date' column
data['Year'] = pd.to_datetime(data['Date']).dt.year
data['Month'] = pd.to_datetime(data['Date']).dt.month
data['Day'] = pd.to_datetime(data['Date']).dt.day

In [7]:
# # Calculate the difference between max and min temperature
# data['TempDiff'] = data['MaxTemp'] - data['MinTemp']

In [8]:
# Perform imputation for missing values in features
imputer = SimpleImputer(strategy='mean')
X_imputed = imputer.fit_transform(X)

# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_imputed, y, test_size=0.2, random_state=42)

In [9]:
# Train Model without through pre-processing
# Initialize Logistic Regression function
model = LogisticRegression(max_iter=1000)

# Train the model
model.fit(X_train, y_train)

# Make predictions on the test set
y_pred = model.predict(X_test)

# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
print("******Without Data Pre-processing***********")
print("Logistic Regression Accuracy on Test Set without Pre-processing: {:.2f}%".format(accuracy * 100))

******Without Data Pre-processing***********
Logistic Regression Accuracy on Test Set without Pre-processing: 76.77%


C:\Users\Vidhya\anaconda3\Lib\site-packages\sklearn\linear_model\_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=2):
ABNORMAL_TERMINATION_IN_LNSRCH.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [10]:
#Kaustav
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier

# Load the dataset
file_path = 'C:/Users/Vidhya/Desktop/Study/Semester 1/ML/Assignments/weatherAUS.csv'
df = pd.read_csv(file_path)

# Dropping columns with more than 40% missing data
df_dropped = df.drop(columns=['Evaporation', 'Sunshine', 'Cloud9am', 'Cloud3pm'])

# Imputing missing values for numerical columns using median
numerical_columns = df_dropped.select_dtypes(include=['float64']).columns
df_dropped[numerical_columns] = df_dropped[numerical_columns].fillna(df_dropped[numerical_columns].median())

# Imputing missing values for categorical columns using mode
categorical_columns = df_dropped.select_dtypes(include=['object']).columns
df_dropped[categorical_columns] = df_dropped[categorical_columns].fillna(df_dropped[categorical_columns].mode().iloc[0])

# Handling skewness by applying log transformation to highly skewed columns
df_dropped['Rainfall'] = np.log1p(df_dropped['Rainfall'])  # log1p is used to avoid log(0)
df_dropped['WindGustSpeed'] = np.log1p(df_dropped['WindGustSpeed'])

# Encoding categorical variables using one-hot encoding
df_encoded = pd.get_dummies(df_dropped, drop_first=True)

# Separating features and target variable
if 'RainTomorrow_Yes' in df_encoded.columns:
    X = df_encoded.drop('RainTomorrow_Yes', axis=1)
    y = df_encoded['RainTomorrow_Yes']
else:
    X = df_encoded.drop('RainTomorrow', axis=1)
    y = df_encoded['RainTomorrow']

# Standardization
numerical_cols = X.select_dtypes(include=['float64', 'int64']).columns
scaler = StandardScaler()
X_scaled = X.copy()
X_scaled[numerical_cols] = scaler.fit_transform(X[numerical_cols])

# Feature Importance using Random Forest
rf = RandomForestClassifier(n_estimators=100, random_state=42)
rf.fit(X_scaled, y)
importances = rf.feature_importances_
feature_names = X_scaled.columns

# Create a dataframe for feature importances
feature_importances = pd.DataFrame({'Feature': feature_names, 'Importance': importances})
feature_importances = feature_importances.sort_values(by='Importance', ascending=False)

# Selecting top 20 features
top_features = feature_importances.head(20)['Feature'].tolist()
X_selected = X_scaled[top_features]

# Splitting the data into training and testing sets (80% train, 20% test)
X_train, X_test, y_train, y_test = train_test_split(X_selected, y, test_size=0.2, random_state=42)

# # Combining the cleaned and selected features with the target variable for final output
# final_df_train = pd.concat([X_train, y_train.reset_index(drop=True)], axis=1)
# final_df_test = pd.concat([X_test, y_test.reset_index(drop=True)], axis=1)

# # Saving the cleaned datasets to CSV files
# train_file_path = 'cleaned_weather_train.csv'
# test_file_path = 'cleaned_weather_test.csv'

# final_df_train.to_csv(train_file_path, index=False)
# final_df_test.to_csv(test_file_path, index=False)

# print(f'Train file saved to {train_file_path}')
# print(f'Test file saved to {test_file_path}')


In [26]:
# Train the Logistic Regression model
# Initialize Logistic Regression function
model = LogisticRegression()

# Train the model
model.fit(X_train, y_train)

# Make predictions on the test set
y_pred = model.predict(X_test)

# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
conf_matrix = confusion_matrix(y_test, y_pred)
class_report = classification_report(y_test, y_pred)
print("Logistic Regression Accuracy on Test Set after Preprocessing: {:.2f}%".format(accuracy * 100))

Logistic Regression Accuracy on Test Set after Preprocessing: 84.33%


In [27]:
print("******After Hyperparameters Tuning***********")
# Define the hyperparameter grid
param_grid_log_reg = {'C': [0.01, 0.1, 1, 10, 100], # Inverse of regularization strength
                      'penalty': ['l2'],   # Type of regularization
                      'solver': ['newton-cg', 'lbfgs', 'liblinear', 'saga'],  # Optimization algorithm
                      'max_iter': [100, 200, 300]                      # Maximum number of iterations
                     }
    
# Set up GridSearchCV
grid_search_log_reg = GridSearchCV(estimator=model, param_grid=param_grid_log_reg, cv=5, scoring='accuracy',error_score='raise')

# Fit the grid search
grid_search_log_reg.fit(X_train, y_train)    

# Best hyperparameters
print("Best hyperparameters for Logistic Regression:", grid_search_log_reg.best_params_)
print("Best score for Logistic Regression: {:.2f}%".format(grid_search_log_reg.best_score_ *100))

# Evaluate on test set
best_log_reg = grid_search_log_reg.best_estimator_
y_pred_log_reg = best_log_reg.predict(X_test)
print("Logistic Regression Accuracy on Test Set after tuning: {:.2f}%".format(accuracy_score(y_test, y_pred_log_reg)*100))


******After Hyperparameters Tuning***********
Best parameters for Logistic Regression: {'C': 0.1, 'max_iter': 100, 'penalty': 'l2', 'solver': 'liblinear'}
Best score for Logistic Regression: 84.80%
Logistic Regression Accuracy on Test Set after tuning: 84.32%


In [28]:
#********* After tuning
model = LogisticRegression(C= 0.1, max_iter = 100, penalty =  'l2', solver = 'liblinear')

# Train the model
model.fit(X_train, y_train)

# Make predictions on the test set
y_pred = model.predict(X_test)

# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
conf_matrix = confusion_matrix(y_test, y_pred)
class_report = classification_report(y_test, y_pred)
print("****** After Hyperparameters identification - Retraining Model***********")
print("Logistic Regression Accuracy on Test Set after Tuning : {:.2f}%".format(accuracy * 100))
print("Confusion Matrix:")
print(conf_matrix)
print("Classification Report:")
print(class_report)


****** After Hyperparameters identification - Retraining Model***********
Logistic Regression Accuracy on Test Set after : 84.32%
Confusion Matrix:
[[4475  251]
 [ 690  584]]
Classification Report:
              precision    recall  f1-score   support

       False       0.87      0.95      0.90      4726
        True       0.70      0.46      0.55      1274

    accuracy                           0.84      6000
   macro avg       0.78      0.70      0.73      6000
weighted avg       0.83      0.84      0.83      6000



In [30]:
# Initialize the Decision Tree Classifier
model_tree = DecisionTreeClassifier()

# Train the model
model_tree.fit(X_train, y_train)

# Make predictions
y_pred = model_tree.predict(X_test)

# Evaluate the model
accuracy_tree = accuracy_score(y_test, y_pred)
conf_matrix_tree = confusion_matrix(y_test, y_pred)
class_report_tree = classification_report(y_test, y_pred)

print("Decision Tree Accuracy on Test Set: {:.2f}%".format(accuracy_tree * 100))


Decision Tree Accuracy on Test Set: 77.47%


In [32]:
print("******After Hyperparameters Tuning***********")
# Define the hyperparameter grid
param_grid = { 'criterion': ['gini', 'entropy'],            # Function to measure the quality of a split
               'max_depth': [10, 20, 30, 40],         # Maximum depth of the tree
               'min_samples_split': [2, 5, 10],              # Minimum number of samples required to split an internal node
               'min_samples_leaf': [1, 2, 5, 10],            # Minimum number of samples required at a leaf node
               'max_features': ['sqrt', 'log2']  # Number of features to consider when looking for the best split
}

# Set up GridSearchCV
grid_search_decision_tree = GridSearchCV(estimator=model_tree, param_grid=param_grid, cv=5, scoring='accuracy')
grid_search_decision_tree.fit(X_train, y_train)

# Best hyperparameters
print("Best hyperparameters for Decision Tree:", grid_search_decision_tree.best_params_)
print("Best score for Decision Tree: {:.2f}%".format(grid_search_decision_tree.best_score_*100))
    

# Evaluate on test set
best_decision_tree = grid_search_decision_tree.best_estimator_
y_pred_decision_tree = best_decision_tree.predict(X_test)
print("Decision Tree Accuracy on Test Set: {:.2f}%".format(accuracy_score(y_test, y_pred_decision_tree)*100))
print("Confusion Matrix:")
print(conf_matrix_tree)
print("Classification Report:")
print(class_report_tree)

Best parameters for Decision Tree: {'criterion': 'entropy', 'max_depth': 10, 'max_features': 'log2', 'min_samples_leaf': 10, 'min_samples_split': 10}
Best score for Decision Tree: 83.65%
Decision Tree Accuracy on Test Set: 82.07%
Confusion Matrix:
[[4023  703]
 [ 649  625]]
Classification Report:
              precision    recall  f1-score   support

       False       0.86      0.85      0.86      4726
        True       0.47      0.49      0.48      1274

    accuracy                           0.77      6000
   macro avg       0.67      0.67      0.67      6000
weighted avg       0.78      0.77      0.78      6000



In [ ]:
# Plot the Decision Tree
# plt.figure(figsize=(12,8))
# tree.plot_tree(model_tree, feature_names=None, class_names=None, filled=True)
# plt.show()